In [2]:
import pandas as pd
from os import listdir

In [19]:
files = listdir("Tables Joined")
dfs = []
for file in files:
    if ".csv" in file:
        print(file)
        dfs.append(pd.read_csv(f"Tables Joined/{file}"))

Customer Master anonymized.csv
Facility Master anonymized.csv
Interfacility Orders anonymized.csv
Parcel File.csv


C:\Users\51917\AppData\Local\Temp\ipykernel_44516\1506131525.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(f"Tables Joined/{file}"))


Product Master anonymized.csv


C:\Users\51917\AppData\Local\Temp\ipykernel_44516\1506131525.py:6: DtypeWarning: Columns (8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(f"Tables Joined/{file}"))


Sales Orders anonymized.csv


C:\Users\51917\AppData\Local\Temp\ipykernel_44516\1506131525.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(f"Tables Joined/{file}"))


# Function to standarize City Names
* based on postal code (if they have the same postal code, then we assign the city name to the first instance)
* cleaned up characters in city names (such as ". , -" spaces and digits)

even after cleaning there are still some cities with similar names (we could send this to GPT to define where do they belong)

There's also postal codes that should probably be normalized (based on the city name)

In [135]:
import pandas as pd
import re
from difflib import SequenceMatcher

# Function to clean and standardize city names based on the same postal code
def clean_and_standardize_cities(df, city_field, postal_code_field):
    df[postal_code_field] = df[postal_code_field].astype(str).apply(lambda x: x.replace(" ", ""))
    unique_cities_per_code = df.groupby(postal_code_field)[city_field].first().reset_index()
    df = df.merge(unique_cities_per_code, on=postal_code_field, suffixes=('', '_first'))

    # Define a function to perform the desired text transformations
    def transform_text(text):
        text = re.sub(r'\.', '', text)
        text = re.sub(r'\,', '', text)
        text = re.sub(r'-', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\d+', ' ', text)
        return text.strip()

    # Apply the text transformation function to the city field
    df[city_field + '_first'] = df[city_field + '_first'].astype(str).apply(transform_text)
    #df.drop(columns=[city_field + '_first'], inplace=True)
    
    return df
# Function to find similar city names within the same country
def find_similar_cities(df, city_field, country_field, similarity_threshold=0.8):
    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()

    grouped = df.groupby(country_field)
    similar_cities = []

    for country, group_data in grouped:
        unique_cities = group_data[[city_field + '_first', country_field]].drop_duplicates()
        for i, city1 in enumerate(unique_cities[city_field + '_first']):
            for j, city2 in enumerate(unique_cities[city_field + '_first']):
                if i != j and similar(city1, city2) > similarity_threshold:
                    similar_cities.append((country, city1, city2))

    filtered_cities = []
    for country, city1, city2 in similar_cities:
        country_cities = [c for c in similar_cities if c[0] == country]
        if len(country_cities) > 1:
            filtered_cities.append((country, city1, city2))

    # Print the filtered similar cities
    for country, city1, city2 in filtered_cities:
        print(f"Country: {country},   {city1:<30}, {city2}")
        
    return filtered_cities

# Function to clean and standardize city names and find similar city names within the same country
def process_data(df, city_field, postal_code_field, country_field, similarity_threshold=0.8):
    df = clean_and_standardize_cities(df, city_field, postal_code_field)
    similar_cities = find_similar_cities(df, city_field, country_field, similarity_threshold)
    return df

# Example usage:
# df_customer_master = process_data(df_customer_master, 'DCCITY', 'DCPC', 'DCCTRY', similarity_threshold=0.8)


## Second DataFrame

In [84]:
dfs[1]

,DFMU,DFFACI,DFNAME,DFTYPE,DFCITY,DFPC,DFCTRY,Receipt Warehouse
0,31,ASI01,supplier1,Supplier,BRILHANTISMO V19,MACAU,NaN,NaN
1,31,BALLS,supplier2,Supplier,XXX,XXX,NaN,NaN
2,31,CACC,supplier3,Supplier,XXX,XXX,NaN,NaN
3,31,CBAG,supplier4,Supplier,XXX,XXX,NaN,NaN
4,31,CCLUB,supplier5,Supplier,XXX,XXX,NaN,NaN
...,...,...,...,...,...,...,...,...
3885,44,42,Warehouse GB person 31,Warehouse,NaN,NaN,GB,Warehouse GB
3886,44,43,Warehouse GB person 32,Warehouse,NaN,NaN,GB,Warehouse GB
3887,44,44,Warehouse GB person 33,Warehouse,NaN,NaN,GB,Warehouse GB
3888,44,45,Warehouse GB person 34,Warehouse,NaN,NaN,GB,Warehouse GB


In [88]:
dfs[1].head()

,DFMU,DFFACI,DFNAME,DFTYPE,DFCITY,DFPC,DFCTRY,Receipt Warehouse
0,31,ASI01,supplier1,Supplier,BRILHANTISMO V19,MACAU,NaN,NaN
1,31,BALLS,supplier2,Supplier,XXX,XXX,NaN,NaN
2,31,CACC,supplier3,Supplier,XXX,XXX,NaN,NaN
3,31,CBAG,supplier4,Supplier,XXX,XXX,NaN,NaN
4,31,CCLUB,supplier5,Supplier,XXX,XXX,NaN,NaN


In [130]:
df_1 = process_data(dfs[0], 'DCCITY', 'DCPC', 'DCCTRY', similarity_threshold=0.85)

Country: AT,   FELDKIRCH                     , FELDKIRCHEN
Country: AT,   LIENZ                         , LINZ
Country: AT,   LEOBENDORF                    , LEOBERSDORF
Country: AT,   LEOBERSDORF                   , LEOBENDORF
Country: AT,   FELDKIRCHEN                   , FELDKIRCH
Country: AT,   LINZ                          , LIENZ
Country: CZ,   HRADEC KRALOVE                , HRADEC KRALOVÉ
Country: CZ,   HRADEC KRALOVÉ                , HRADEC KRALOVE
Country: DE,   VELLINGE                      , ELLINGEN
Country: DE,   DÜSSELDORF                    , DUESSELDORF
Country: DE,   SOLINGEN                      , LINGEN
Country: DE,   HILDESHEIM                    , HILLESHEIM
Country: DE,   GERNSHEIM                     , IGERSHEIM
Country: DE,   PLEISKIRCHEN                  , REISKIRCHEN
Country: DE,   ROHRBACH                      , ROEHRNBACH
Country: DE,   FRANKFURT                     , FRANKFURT Y
Country: DE,   REISKIRCHEN                   , PLEISKIRCHEN
Country: DE,   LAU

In [126]:
df_1.head()

,DCMU,DCSTKY,DCNAME,DCCITY,DCPC,DCCTRY,DCCTYP new,DCMSEG,DCCRDT,DCCITY_first
0,30,DCACU1,Client A COMPANY,FAIRHAVEN,MA02719,US,CROSS MARKET CUSTOMER,CROSSMARKET,No credit code assigned,FAIRHAVEN
1,31,DCACU1,Client A COMPANY,FAIRHAVEN,MA02719,US,CROSS MARKET CUSTOMER,CROSSMARKET,No credit code assigned,FAIRHAVEN
2,43,439003,XY BRAND A SPORT DIVISION,FAIRHAVEN,MA02719,AT,SPORT ALLOWANCE,PROMOTIONS,Hold if 15 days past,FAIRHAVEN
3,30,DRP33,Client A FRANCE SA,MONTATAIRE CEDEX,60765,FR,CROSS MARKET CUSTOMER,CROSSMARKET,No credit code assigned,MONTATAIRE CEDEX
4,30,MU33,Client A FRANCE SA,MONTATAIRE CEDEX,60765,FR,CROSS MARKET CUSTOMER,CROSSMARKET,No credit code assigned,MONTATAIRE CEDEX


In [127]:
# compare the amount of different city names before and after cleaning
df_1.DCCITY.unique().shape,df_1.DCCITY_first.unique().shape

((6013,), (4664,))

In [136]:
df_2 = process_data(dfs[1], 'DFCITY', 'DFPC', 'DFCTRY', similarity_threshold=0.85)

Country: GB,   TELFORD SHROPSHIRE            , TELFORDSHROPSHIRE
Country: GB,   SPALDING LINCS                , SPALDINGLINCS
Country: GB,   DUDLEYW MIDLANDS              , DUDLEY W MIDLANDS
Country: GB,   HUNTINGDON CAMBS              , HUNTINGDONCAMBS
Country: GB,   HUNTINGDON CAMBS              , HUNTINGDON CAMS
Country: GB,   WEST YORKSHIRE                , EAST YORKSHIRE
Country: GB,   WEST YORKSHIRE                , STH YORKSHIRE
Country: GB,   HUNTINGDONCAMBS               , HUNTINGDON CAMBS
Country: GB,   HUNTINGDONCAMBS               , HUNTINGDON CAMS
Country: GB,   EAST YORKSHIRE                , WEST YORKSHIRE
Country: GB,   EAST YORKSHIRE                , STH YORKSHIRE
Country: GB,   WARRINGTON                    , WERRINGTON
Country: GB,   NR WESTERHAM KENT             , WESTERHAM KENT
Country: GB,   NORTH YORKSHIRE               , STH YORKSHIRE
Country: GB,   NORTH YORKSHIRE               , SOUTH YORKSHIRE
Country: GB,   WORCESTERSHIRE                , WORCESTSHIRE
Countr

In [128]:
# compare the amount of different city names before and after cleaning
df_2.DFCITY.unique().shape,df_2.DFCITY_first.unique().shape

((1348,), (1011,))

## Third dataframe

In [103]:
dfs[2].head()

,DIMU,DIFFAC,DIIOID,DITYPE,DITFAC,DIPART,DIQTY,DIIUM,DIRDAT
0,31,DCACU4,41156.0,Purchase order,NL,SKU21125,24,PR,220103
1,31,DCACU4,41156.0,Purchase order,NL,SKU21127,12,PR,220103
2,31,DCACU4,41156.0,Purchase order,NL,SKU21129,36,PR,220103
3,31,DCACU4,41156.0,Purchase order,NL,SKU21131,24,PR,220103
4,31,DCACU4,41156.0,Purchase order,NL,SKU21133,48,PR,220103


## Fourth dataframe

In [104]:
dfs[3].head()

,DAMU,DAORD#,DASUFX,DACARR,DAPARN,DACTRY,DAFRTT,DASERC,DASERD
0,31,106457,2,NLD,5228916320361,FR,XWK,101.0,NaN
1,31,106457,2,NLD,5228916320361,FR,XWK,101.0,NaN
2,31,106457,2,NLD,5228916320361,FR,XWK,101.0,NaN
3,31,112788,1,NLD,5228916309387,FR,XWK,101.0,NaN
4,31,112788,1,NLD,5228916309387,FR,XWK,101.0,NaN


## Fifth dataframe

In [105]:
dfs[4].head()

,DPMU,DPWH,DPPART,DPIUM,DPPUM,DPSUM,DPSAF,DPPLIN,DPPCLS,DPDESC,DPBRND,DPDIVN,DPPSTR,DPUSR3,DPVAS,DPCORG,DPWGTK,DPCOMV,DPCOGS,DPCURC
0,44,UK,SKU68030,EA,EA,EA,1.0,85,50,SKU68030,Brand A ACCESSORIES,NaN,VZAAAAA,C,NaN,NaN,0.200,NaN,19.3,£
1,44,UK,SKU68033,EA,EA,EA,1.0,SA,99,SKU68033,CONSUMABLES,NaN,DDAAAAA,C,NaN,NaN,0.012,NaN,0.0,£
2,44,UK,SKU68034,EA,EA,EA,1.0,SA,99,SKU68034,CONSUMABLES,NaN,DDAAAAA,C,NaN,NaN,0.012,NaN,0.0,£
3,44,UK,SKU68036,EA,EA,EA,1.0,SA,99,SKU68036,CONSUMABLES,NaN,DDAAAAA,C,NaN,NaN,0.413,NaN,0.0,£
4,44,UK,SKU68037,EA,EA,EA,1.0,SA,99,SKU68037,CONSUMABLES,NaN,DDAAAAA,C,NaN,NaN,0.032,NaN,0.0,£


## Sixth dataframe

In [124]:
dfs[5].head()

,DOMU,DOWH,DOWHD,DOSTKY,DOSTMU,DOORD#,DOSUFX,DOPART,DOQTY,DOSUM,DOORIG,DODTYP,DODYMD,DOSVAL,DOCURC,DOCUST,VALUE €
0,31,NL,WarehouseNL1,312160,30,100068,2,SKU71623,1,EA,NaN,NaN,220411,0.0075,EU,NaN,0.006919
1,31,NL,WarehouseNL1,334000,33,100076,0,SKU45337,1,PR,NaN,NaN,220113,16.3875,EU,NaN,15.117469
2,31,NL,WarehouseNL1,32A600,33,100132,0,SKU41144,2,EA,OASIS,Y,220304,24.1425,EU,NaN,22.271456
3,31,NL,WarehouseNL1,32A600,33,100132,0,SKU41145,1,EA,OASIS,Y,220304,12.0750,EU,NaN,11.139188
4,31,NL,WarehouseNL1,32A600,33,100132,0,SKU41151,1,EA,OASIS,Y,220304,12.0750,EU,NaN,11.139188


In [144]:
import re
from difflib import SequenceMatcher
from unicodedata import normalize

# Improved function to clean and standardize city names based on the same postal code
def clean_and_standardize_cities(df, city_field, postal_code_field):
    df[postal_code_field] = df[postal_code_field].astype(str).apply(lambda x: x.replace(" ", "").upper())

    # Normalize postal codes to handle different formats
    df[postal_code_field] = df[postal_code_field].apply(lambda x: re.sub(r'\D', '', x))

    unique_cities_per_code = df.groupby(postal_code_field)[city_field].first().reset_index()
    df = df.merge(unique_cities_per_code, on=postal_code_field, suffixes=('', '_first'))

    def transform_text(text):
        # Normalize the text for consistent comparison: remove accents, lowercase, and strip spaces
        text = normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii').lower()
        text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic characters
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        return text.strip()

    df[city_field + '_first'] = df[city_field + '_first'].astype(str).apply(transform_text)
    
    return df

# Function to find similar city names within the same country with enhancements
def find_similar_cities(df, city_field, country_field, similarity_threshold=0.8):
    def similar(a, b):
        # Handle short city names differently
        if len(a) < 5 or len(b) < 5:
            return a == b
        return SequenceMatcher(None, a, b).ratio()

    grouped = df.groupby(country_field)
    similar_cities = []

    for country, group_data in grouped:
        unique_cities = group_data[[city_field + '_first', country_field]].drop_duplicates()
        for i, city1 in enumerate(unique_cities[city_field + '_first']):
            for j, city2 in enumerate(unique_cities[city_field + '_first']):
                if i != j and similar(city1, city2) > similarity_threshold:
                    similar_cities.append((country, city1, city2))

    return similar_cities

# Updated function to process data with the new enhancements
def process_data(df, city_field, postal_code_field, country_field, similarity_threshold=0.85):
    df = clean_and_standardize_cities(df, city_field, postal_code_field)
    similar_cities = find_similar_cities(df, city_field, country_field, similarity_threshold)
    return df, similar_cities

# Applying the enhanced process_data function
processed_df, similar_cities = process_data(dfs[1], 'DFCITY', 'DFPC', 'DFCTRY', similarity_threshold=0.85)

# Displaying a few similar city pairs for review
similar_cities 

[('GB', 'chesire', 'cheshire'),
 ('GB', 'swindon', 'swinton'),
 ('GB', 'west yorkshire', 'east yorkshire'),
 ('GB', 'west yorkshire', 'sth yorkshire'),
 ('GB', 'huntingdoncambs', 'huntingdon cambs'),
 ('GB', 'east yorkshire', 'west yorkshire'),
 ('GB', 'east yorkshire', 'sth yorkshire'),
 ('GB', 'east lothian', 'west lothian'),
 ('GB', 'cheshire', 'chesire'),
 ('GB', 'huntingdon cambs', 'huntingdoncambs'),
 ('GB', 'west lothian', 'east lothian'),
 ('GB', 'swinton', 'swindon'),
 ('GB', 'sth yorkshire', 'west yorkshire'),
 ('GB', 'sth yorkshire', 'east yorkshire'),
 ('GB', 'sth yorkshire', 'south yorkshire'),
 ('GB', 'south yorkshire', 'sth yorkshire'),
 ('NL', 'amsterdam', 'amsterdam zo'),
 ('NL', 'amsterdam zo', 'amsterdam')]

In [142]:
processed_df, similar_cities = process_data(dfs[0], 'DCCITY', 'DCPC', 'DCCTRY', similarity_threshold=0.85)
similar_cities

[('AT', 'amsterdam', 'amsterdam kj'),
 ('AT', 'feldkirch', 'feldkirchen'),
 ('AT', 'feldkirchen', 'feldkirch'),
 ('AT', 'amsterdam kj', 'amsterdam'),
 ('BE', 'knokkeheist', 'knokke heist'),
 ('BE', 'knokke heist', 'knokkeheist'),
 ('DE', 'vellinge', 'ellingen'),
 ('DE', 'guetersloh', 'gutersloh'),
 ('DE', 'dusseldorf', 'duesseldorf'),
 ('DE', 'solingen', 'lingen'),
 ('DE', 'hildesheim', 'hillesheim'),
 ('DE', 'gernsheim', 'igersheim'),
 ('DE', 'pleiskirchen', 'reiskirchen'),
 ('DE', 'rohrbach', 'roehrnbach'),
 ('DE', 'munchen', 'muenchen'),
 ('DE', 'reiskirchen', 'pleiskirchen'),
 ('DE', 'lauterhofen', 'aiterhofen'),
 ('DE', 'muelheim ad ruhr', 'muelheimruhr'),
 ('DE', 'duesseldorf', 'dusseldorf'),
 ('DE', 'bergisch gladbach', 'bergischgladbach'),
 ('DE', 'bergischgladbach', 'bergisch gladbach'),
 ('DE', 'munster', 'muenster'),
 ('DE', 'monchengladbach', 'moenchengladbach'),
 ('DE', 'lohne', 'loehne'),
 ('DE', 'bad tolz', 'bad toelz'),
 ('DE', 'arnsberg', 'marsberg'),
 ('DE', 'nurnberg

In [150]:
def find_similar_cities_enhanced(df, city_field, postal_code_field, country_field, similarity_threshold=0.8):
    def similar(a, b):
        # Basic string similarity
        ratio = SequenceMatcher(None, a, b).ratio()

        # Additional checks for improved accuracy
        len_diff = abs(len(a) - len(b))
        if ratio > similarity_threshold and len_diff <= 3:
            return True
        return False

    grouped = df.groupby(country_field)
    similar_cities = []

    for country, group_data in grouped:
        unique_cities = group_data[[city_field, postal_code_field, country_field]].drop_duplicates()

        # Convert city names and postal codes to strings
        unique_cities[city_field] = unique_cities[city_field].astype(str)
        unique_cities[postal_code_field] = unique_cities[postal_code_field].astype(str)

        for i, row1 in unique_cities.iterrows():
            for j, row2 in unique_cities.iterrows():
                if i != j:
                    city1, postal_code1 = row1[city_field], row1[postal_code_field]
                    city2, postal_code2 = row2[city_field], row2[postal_code_field]

                    # Check if cities are similar and postal codes are significantly different
                    if similar(city1, city2) and postal_code1 != postal_code2:
                        similar_cities.append((country, city1, city2, postal_code1, postal_code2))

    return similar_cities

# Example usage
similar_cities_enhanced = find_similar_cities_enhanced(processed_df, 'DFCITY', 'DFPC', 'DFCTRY', 0.85)


In [151]:
similar_cities_enhanced

[('GB', '.', '.', '', '1'),
 ('GB', '.', '.', '', '39'),
 ('GB', '.', '.', '', '45'),
 ('GB', '.', '.', '', '143'),
 ('GB', '.', '.', '', '15'),
 ('GB', '.', '.', '', '47'),
 ('GB', '.', '.', '', '38'),
 ('GB', '.', '.', '', '2'),
 ('GB', '.', '.', '', '58'),
 ('GB', '.', '.', '', '13'),
 ('GB', '.', '.', '', '35'),
 ('GB', '.', '.', '', '17'),
 ('GB', '.', '.', '', '12'),
 ('GB', '.', '.', '', '981'),
 ('GB', '.', '.', '', '4'),
 ('GB', '.', '.', '', '33'),
 ('GB', '.', '.', '', '366'),
 ('GB', '.', '.', '', '41'),
 ('GB', '.', '.', '', '133'),
 ('GB', '.', '.', '', '826'),
 ('GB', '.', '.', '', '14'),
 ('GB', '.', '.', '', '36'),
 ('GB', '.', '.', '', '16'),
 ('GB', '.', '.', '', '233'),
 ('GB', '.', '.', '', '377'),
 ('GB', '.', '.', '', '990'),
 ('GB', 'PETERBOROUGH', 'PETERBOROUGH', '', '11'),
 ('GB', 'PETERBOROUGH', 'PETERBOROUGH', '', '27'),
 ('GB', 'PETERBOROUGH', 'PETERBOROUGH', '', '45'),
 ('GB', 'PETERBOROUGH', 'PETERBOROUGH', '', '15'),
 ('GB', 'PETERBOROUGH', 'PETERBOROUGH